In [3]:
import pandas as pd

# URL of the CSV file
url = "https://raw.githubusercontent.com/GuviMentor88/Training-Datasets/refs/heads/main/twitter_training.csv"

# Directly read the CSV from the URL into a pandas DataFrame
df = pd.read_csv(url)

# Show the first few rows of the DataFrame
print(df.head())



   2401  Borderlands  Positive  \
0  2401  Borderlands  Positive   
1  2401  Borderlands  Positive   
2  2401  Borderlands  Positive   
3  2401  Borderlands  Positive   
4  2401  Borderlands  Positive   

  im getting on borderlands and i will murder you all ,  
0  I am coming to the borders and I will kill you...     
1  im getting on borderlands and i will kill you ...     
2  im coming on borderlands and i will murder you...     
3  im getting on borderlands 2 and i will murder ...     
4  im getting into borderlands and i can murder y...     


In [4]:
# Define new column names
columns = ["Tweet_ID", "Entity", "Sentiment", "Text_Content"]
# Assign the new column names to the DataFrame
df.columns = columns
# Display the first few rows of the updated DataFrame
print(df.head())



   Tweet_ID       Entity Sentiment  \
0      2401  Borderlands  Positive   
1      2401  Borderlands  Positive   
2      2401  Borderlands  Positive   
3      2401  Borderlands  Positive   
4      2401  Borderlands  Positive   

                                        Text_Content  
0  I am coming to the borders and I will kill you...  
1  im getting on borderlands and i will kill you ...  
2  im coming on borderlands and i will murder you...  
3  im getting on borderlands 2 and i will murder ...  
4  im getting into borderlands and i can murder y...  


In [6]:
df["Sentiment"].value_counts()

,count
Sentiment,
Negative,22542
Positive,20831
Neutral,18318
Irrelevant,12990


In [5]:
import pandas as pd

# Assuming your DataFrame is named df
nan_check = df.isna().sum()
print(nan_check)


Tweet_ID          0
Entity            0
Sentiment         0
Text_Content    686
dtype: int64


In [6]:
df_without_na = df.dropna(subset=['Text_Content'])


In [7]:
df_without_na.shape

(73995, 4)

In [8]:
!pip install transformers datasets torch scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [9]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

In [10]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [11]:
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

# 1. Clean the entire dataset
df_without_na['Text_Content'] = df_without_na['Text_Content'].str.lower()
df_without_na['Text_Content'] = df_without_na['Text_Content'].str.replace('[^\w\s]', '', regex=True)
df_without_na['Text_Content'] = df_without_na['Text_Content'].str.replace(r'\s+', ' ', regex=True).str.strip()

# Remove stopwords
stop_words = set(stopwords.words('english'))
df_without_na['Text_Content'] = df_without_na['Text_Content'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))

# Apply stemming (or lemmatization)
stemmer = PorterStemmer()
df_without_na['Text_Content'] = df_without_na['Text_Content'].apply(lambda x: ' '.join([stemmer.stem(word) for word in x.split()]))

<ipython-input-11-4dc49521c0f2>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_without_na['Text_Content'] = df_without_na['Text_Content'].str.lower()
<ipython-input-11-4dc49521c0f2>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_without_na['Text_Content'] = df_without_na['Text_Content'].str.replace('[^\w\s]', '', regex=True)
<ipython-input-11-4dc49521c0f2>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

In [12]:
df_without_na["Text_Content"].head()

,Text_Content
0,come border kill
1,im get borderland kill
2,im come borderland murder
3,im get borderland 2 murder
4,im get borderland murder


In [13]:
from sklearn.model_selection import train_test_split

train_texts, val_texts, train_labels, val_labels = train_test_split(
    df_without_na['Text_Content'],
    df_without_na['Sentiment'],
    test_size=0.2,
    random_state=42)

# Display sizes of the splits
print("Training set size:", len(train_texts))
print("Validation set size:", len(val_texts))


Training set size: 59196
Validation set size: 14799


In [14]:
print(type(train_texts))    # Check the first few entries


<class 'pandas.core.series.Series'>


In [15]:
# Tokenize the data
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Convert train_texts and val_texts to lists of strings
train_texts_list = train_texts.tolist()  # Convert pandas Series to list
val_texts_list = val_texts.tolist()  # Convert pandas Series to list


# Tokenize the data with truncation and padding
train_encodings = tokenizer(train_texts_list, truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(val_texts_list, truncation=True, padding=True, max_length=128)




/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [17]:
train_texts_list[0]

'dream girlfriend got marri rthen drive home fuckin runaway bu hippi dude montag buse block 3 lane race hill road fuckin blow thru shit like grand theft auto'

In [26]:
train_encodings[0]

Encoding(num_tokens=128, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [18]:
# Create a mapping from sentiment labels to numerical values
sentiment_mapping = {
    'Negative': 0,
    'Neutral': 1,
    'Positive': 2,
    'Irrelevant': 3 # If the dataset has this key
}

# Convert labels to numerical values
train_labels_numerical = [sentiment_mapping.get(label, -1) for label in train_labels]
val_labels_numerical = [sentiment_mapping.get(label, -1) for label in val_labels]

# Optional: Print unique numerical labels to check
print(set(train_labels_numerical))
print(set(val_labels_numerical))

{0, 1, 2, 3}
{0, 1, 2, 3}


In [19]:
# Create a PyTorch Dataset
class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

# Convert labels to numerical values using the sentiment_mapping and aligning with the cleaned texts
train_labels_numerical = [sentiment_mapping.get(label, -1) for i, label in enumerate(train_labels) if i < len(train_texts)]
val_labels_numerical = [sentiment_mapping.get(label, -1) for i, label in enumerate(val_labels) if i < len(val_texts)]

# Create datasets with numerical labels
train_dataset = SentimentDataset(train_encodings, train_labels_numerical)
val_dataset = SentimentDataset(val_encodings, val_labels_numerical)

In [20]:
# Load the model and move to CPU
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=4  # Assuming 4 sentiment classes: Negative, Neutral, Positive , Irrelevant
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
num_labels = len(sentiment_mapping)
print(num_labels)

4


In [22]:
# Import necessary libraries
from transformers import EarlyStoppingCallback, Trainer, TrainingArguments

In [23]:
# Define training arguments with GPU and increased batch size
training_args = TrainingArguments(
    output_dir="./results",            # Directory to save model checkpoints
    num_train_epochs= 5,               # Max number of epochs
    per_device_train_batch_size=32,    # Batch size for training
    per_device_eval_batch_size=64,     # Batch size for evaluation
    warmup_steps=500,                  # Warmup steps for learning rate scheduler
    weight_decay=0.01,                 # Weight decay for regularization
    logging_dir="./logs",              # Directory for logging
    evaluation_strategy="epoch",       # Evaluate at the end of each epoch
    save_strategy="epoch",             # Save model at the end of each epoch
    save_total_limit=2,                # Keep only the 2 latest checkpoints
    load_best_model_at_end=True,       # Load the best model at the end of training
    logging_steps=100,
    report_to="none",                  # Disable reporting for now
    fp16=True                          # Enable mixed precision for faster training on GPU
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [24]:
# Ensure the model and data are moved to GPU
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("Using CPU. Consider using a GPU for faster training.")

Using GPU: Tesla T4


In [25]:
import torch
print(torch.cuda.is_available())  # Should return True
print(torch.cuda.get_device_name(0))  # Should return Tesla T4


True
Tesla T4


In [26]:
# Move the model to the selected device
model = model.to(device)

In [27]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(p):
    logits, labels = p
    predictions = logits.argmax(axis=-1)  # Convert logits to predicted class
    accuracy = accuracy_score(labels, predictions)

    # Handle zero-division warning by setting zero_division to 1
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, predictions, average='weighted', zero_division=1
    )
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }


In [28]:
# Define Trainer with EarlyStoppingCallback
trainer = Trainer(
    model=model,                       # The model
    args=training_args,                # Training arguments
    train_dataset=train_dataset,       # Training dataset
    eval_dataset=val_dataset,          # Validation dataset
    tokenizer=tokenizer,               # Tokenizer
    compute_metrics=compute_metrics,   # Metrics function
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]  # Stop if no improvement after 2 epochs
)

<ipython-input-28-88c4a5437795>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [29]:
# Train the model
trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.690600,0.629960,0.762889,0.768180,0.762889,0.759247
2,0.324900,0.368879,0.869586,0.871286,0.869586,0.869318
3,0.191200,0.338144,0.891006,0.891585,0.891006,0.890990
4,0.132700,0.351876,0.905264,0.906311,0.905264,0.905285
5,0.078500,0.384741,0.910129,0.911216,0.910129,0.910284


TrainOutput(global_step=9250, training_loss=0.34871889217479807, metrics={'train_runtime': 1158.9271, 'train_samples_per_second': 255.391, 'train_steps_per_second': 7.982, 'total_flos': 9802274770391040.0, 'train_loss': 0.34871889217479807, 'epoch': 5.0})

In [30]:
# Evaluate the model
results = trainer.evaluate()
print("Validation results:", results)

Validation results: {'eval_loss': 0.3381436765193939, 'eval_accuracy': 0.891006149064126, 'eval_precision': 0.891585468102018, 'eval_recall': 0.891006149064126, 'eval_f1': 0.8909900663579915, 'eval_runtime': 16.8455, 'eval_samples_per_second': 878.515, 'eval_steps_per_second': 13.772, 'epoch': 5.0}


In [31]:
# Stop training if you're done
trainer.save_model("/content/drive/MyDrive/Distilbert-base-uncased")  # Save model
tokenizer.save_pretrained("/content/drive/MyDrive/Distilbert-base-uncased")  # Save tokenizer

('/content/drive/MyDrive/Distilbert-base-uncased/tokenizer_config.json',
 '/content/drive/MyDrive/Distilbert-base-uncased/special_tokens_map.json',
 '/content/drive/MyDrive/Distilbert-base-uncased/vocab.txt',
 '/content/drive/MyDrive/Distilbert-base-uncased/added_tokens.json',
 '/content/drive/MyDrive/Distilbert-base-uncased/tokenizer.json')

In [32]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
import torch.nn.functional as F

# Load fine-tuned model and tokenizer
model_path = "/content/drive/MyDrive/Distilbert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [33]:
def predict_sentiment(texts):
    # Tokenize input texts
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=128)

    # Get model predictions
    with torch.no_grad():
        outputs = model(**inputs)

    # Apply softmax to get probabilities
    probs = F.softmax(outputs.logits, dim=-1)

    # Get class probabilities and format the output
    results = []
    for i, prob in enumerate(probs):
        result = {
            "Text": texts[i],
            "Probabilities": {
                "Negative": round(prob[0].item(), 4),
                "Neutral": round(prob[1].item(), 4),
                "Positive": round(prob[2].item(), 4),
            }
        }
        results.append(result)
    return results

In [35]:
texts = [
    "I love this product!",
    "This is the worst experience ever.",
    "I got a very decent hike to support my needs"
]

predictions = predict_sentiment(texts)

# Display results
for prediction in predictions:
    print(f"Text: {prediction['Text']}")
    for sentiment, prob in prediction["Probabilities"].items():
        print(f"  {sentiment}: {prob}")
    print()


Text: I love this product!
  Negative: 0.0016
  Neutral: 0.001
  Positive: 0.9941

Text: This is the worst experience ever.
  Negative: 0.9964
  Neutral: 0.0005
  Positive: 0.0016

Text: I got a very decent hike to support my needs
  Negative: 0.0154
  Neutral: 0.2768
  Positive: 0.6918



In [1]:
!pip install gradio

In [2]:
#!pip install gradio flask

import gradio as gr
import torch
import torch.nn.functional as F
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from flask import request
import logging
from datetime import datetime

# Load fine-tuned model and tokenizer
model_path = "/content/drive/MyDrive/Distilbert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Set up logging
logging.basicConfig(
    filename="user_logs.txt",  # Logs will be saved to this file
    level=logging.INFO,
    format="%(asctime)s - %(message)s",
)

def log_user_details(user_ip, input_text):
    log_message = f"IP: {user_ip} | Input: {input_text}"
    logging.info(log_message)
    print(log_message)  # Optional: Print logs for testing in Colab

def predict_sentiment(text):
    # Get user IP address (only works with Flask requests)
    user_ip = request.remote_addr if request else "Unknown IP"
    log_user_details(user_ip, text)

    # Predict sentiment
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)
    probs = F.softmax(outputs.logits, dim=-1)
    # Get class probabilities and format the output
    result = {
        "Negative": round(probs[0][0].item(), 4),
        "Neutral": round(probs[0][1].item(), 4),
        "Positive": round(probs[0][2].item(), 4),
    }
    return result

iface = gr.Interface(
  fn=predict_sentiment,
  inputs=gr.Textbox(lines=2, placeholder="Enter text here..."),
  outputs=gr.JSON(),
  title="Sentiment Analysis",
  description="Enter some text and get the predicted sentiment probabilities."
)

iface.launch(share=True)  # `share=True` creates a public URL


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5a2ea4b12ca2d516da.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [3]:
# Read the log file and print its content
log_file_path = "user_logs.txt"

try:
    with open(log_file_path, "r") as log_file:
        logs = log_file.readlines()
        print("User Logs:")
        for line in logs:
            print(line.strip())  # Remove extra newline characters
except FileNotFoundError:
    print(f"Log file '{log_file_path}' not found.")


Log file 'user_logs.txt' not found.
